In [85]:
import numpy as np
import matplotlib.pyplot
import pandas as pd
import math
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import LabelEncoder

In [86]:
mushroom = fetch_ucirepo(id=73) 
mushroom_features = mushroom.data.features
mushroom_targets = mushroom.data.targets

In [87]:
print(mushroom_targets)

     poisonous
0            p
1            e
2            e
3            p
4            e
...        ...
8119         e
8120         e
8121         e
8122         p
8123         e

[8124 rows x 1 columns]


In [88]:
mushroom_features

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,x,s,n,t,p,f,c,n,k,e,...,s,w,w,p,w,o,p,k,s,u
1,x,s,y,t,a,f,c,b,k,e,...,s,w,w,p,w,o,p,n,n,g
2,b,s,w,t,l,f,c,b,n,e,...,s,w,w,p,w,o,p,n,n,m
3,x,y,w,t,p,f,c,n,n,e,...,s,w,w,p,w,o,p,k,s,u
4,x,s,g,f,n,f,w,b,k,t,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,s,o,o,p,o,o,p,b,c,l
8120,x,s,n,f,n,a,c,b,y,e,...,s,o,o,p,n,o,p,b,v,l
8121,f,s,n,f,n,a,c,b,n,e,...,s,o,o,p,o,o,p,b,c,l
8122,k,y,n,f,y,f,c,n,b,t,...,k,w,w,p,w,o,e,w,v,l


In [89]:
features_to_encode = mushroom_features[['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']]
encoded_features = pd.DataFrame()
label_encoder = LabelEncoder()
for column in features_to_encode.columns:
    encoded_features[column] = label_encoder.fit_transform(features_to_encode[column])
features = encoded_features.to_numpy()
classes = list(mushroom_targets['poisonous'].unique())
labels = mushroom_targets['poisonous'].replace(classes, range(0, len(classes))).to_numpy()
y = np.zeros((labels.shape[0], len(classes)))
y[np.arange(labels.shape[0]), labels]= 1

In [90]:
features

array([[5, 2, 4, ..., 2, 3, 5],
       [5, 2, 9, ..., 3, 2, 1],
       [0, 2, 8, ..., 3, 2, 3],
       ...,
       [2, 2, 4, ..., 0, 1, 2],
       [3, 3, 4, ..., 7, 4, 2],
       [5, 2, 4, ..., 4, 1, 2]])

In [91]:
y

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [92]:
input_neurons = features.shape[1]
hidden_neurons = 4
hiddener_neurons = 6
output_neurons = len(classes)

print('Number of input neurons: {}'.format(input_neurons))
print('Number of hidden neurons 1: {}'.format(hidden_neurons))
print('Number of hidden neurons 2: {}'.format(hiddener_neurons))
print('Number of output neurons: {}'.format(output_neurons))

np.random.seed(6615)
# weight between input and hidden
w_ih = np.random.uniform(size=(input_neurons, hidden_neurons))
biases_h = np.zeros((1, hidden_neurons))

# hiddener neurons 
w_hh2 = np.random.uniform(size=(hidden_neurons, hiddener_neurons))
biases_hh2 = np.zeros((1, hiddener_neurons))

w_hh = np.random.uniform(size=(hidden_neurons, hidden_neurons))
biases_hh = np.zeros((1, hidden_neurons))

#weight between hidden and output
w_ho = np.random.uniform(size=(hidden_neurons, output_neurons))
w_ho2 = np.random.uniform(size=(hiddener_neurons, output_neurons))

biases_o = np.zeros((1, output_neurons))
biases_o2 = np.zeros((1, output_neurons))

lr= 0.0085

Number of input neurons: 22
Number of hidden neurons 1: 4
Number of hidden neurons 2: 6
Number of output neurons: 2


In [93]:
# Function to compute the sigmoid activation
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Function to compute the derivative of the sigmoid function
def sigmoid_d(x):
    return x * (1 - x)

# Function for the linear transformation (dot product of input, weights, and addition of bias)
def linear(x, w, b):
    return np.dot(x, w) + b

def forward_pass(x):
    global w_ih, biases_h, w_hh, biases_hh, w_ho, biases_o, w_hh2, biases_hh2, w_ho2, biases_o2
    
    # Hidden layer 1
    hidden_layer_1 = sigmoid(linear(x, w_ih, biases_h))
    
    # Hidden layer 2
    hidden_layer_2 = sigmoid(linear(hidden_layer_1, w_hh, biases_hh))
    
    # New hidden layer (hidden_layer)
    hidden_layer = sigmoid(linear(hidden_layer_2, w_hh2, biases_hh2))
    
    # Output layer
    output_layer = linear(hidden_layer, w_ho2, biases_o2)
    
    return output_layer, hidden_layer, hidden_layer_2, hidden_layer_1

In [94]:
# Update the number of epochs
num_epochs = 1000

# Training loop
for e in range(num_epochs):
    # Forward pass
    out_out, h_out, h_out_2, h_out_1 = forward_pass(features)
    
    # Compute error
    error = y - out_out
    
    # Backpropagation
    out_err_term = error * sigmoid_d(out_out)
    h_error_2 = out_err_term.dot(w_ho2.T)
    h_err_term_2 = h_error_2 * sigmoid_d(h_out)
    
    h_error_1 = h_err_term_2.dot(w_hh2.T)
    h_err_term_1 = h_error_1 * sigmoid_d(h_out_2)
    
    # Update weights and biases
    w_ho2 += h_out.T.dot(out_err_term) * lr
    biases_o2 += np.sum(out_err_term, axis=0, keepdims=True) * lr
    
    w_hh2 += h_out_2.T.dot(h_err_term_2) * lr
    biases_hh2 += np.sum(h_err_term_2, axis=0, keepdims=True) * lr
    
    w_hh += h_out_1.T.dot(h_err_term_1) * lr
    biases_hh += np.sum(h_err_term_1, axis=0, keepdims=True) * lr
    
    w_ih += features.T.dot(h_err_term_1) * lr
    biases_h += np.sum(h_err_term_1, axis=0, keepdims=True) * lr

C:\Users\zenon\AppData\Local\Temp\ipykernel_23144\2866172182.py:13: RuntimeWarning: overflow encountered in multiply
  out_err_term = error * sigmoid_d(out_out)
C:\Users\zenon\AppData\Local\Temp\ipykernel_23144\2866172182.py:15: RuntimeWarning: invalid value encountered in multiply
  h_err_term_2 = h_error_2 * sigmoid_d(h_out)


In [95]:
def predict(x):
    p_out = linear(x, w_ih, biases_h)
    p_out = sigmoid(p_out)
    p_out = linear(p_out, w_ho, biases_o)
    p_out = sigmoid(p_out)

    return p_out

In [96]:
# Predicting our training data

output = predict(features)
p_labels = np.argmax(output, axis=1)

accuracy = np.mean(p_labels == y.argmax(axis=1))
print('Accuracy: {:0.2f}%'.format(accuracy*100))

Accuracy: 48.20%
